# 텍스트 분류 실습 - 20 뉴스 그룹 분류 
<br>

__< Dataset >__<br>
* Scikit learn 내장 dataset : 20 뉴스그룹 데이터 세트 
<br><br>

__< Classification model >__<br><br>
텍스트를 피처 벡터화로 변환하면 일반적으로 희소 행렬이 형태가 되는데, 희소 행렬에 분류를 효과적으로 잘 처리 하기 위한 알고리즘은 <br>
* 로지스틱 회귀 ⇒ 이거 사용
* 선형 서포트 벡터 머신 = SVM
* 나이브 베이즈  
<br>

__< Process >__<br>
1. 텍스트 전처리<br><br>
2. 피처 벡터화 
    * BOW - Count 기반 , TF-IDF 예측 성능 비교 


3. ML 알고리즘 적용해 분류를 학습 / 예측 / 평가 
    * 피처 벡터화 파라미터와 GridSearchCV 기반의 하이퍼 파라미터 튜닝 
    * Pipeline 객체를 통해 앞 두 파라미터 동시 튜닝

## 1. 데이터 가져오기 & 텍스트 전처리 

In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all',random_state=156)

In [2]:
import pandas as pd 

# 해당 dataset 은 딕셔너리와 유사한 Bunch 객체를 반환하는데 이 때의 key 값 확인 
print("20 news group Dataset - key value : ", news_data.keys(),'\n\n')

# 해당 dataset target 확인 
print('target 클래스의 값과 분포도\n',pd.Series(news_data.target).value_counts().sort_index(),'\n\n')
print('target 클래스의 이름들\n', news_data.target_names)

20 news group Dataset - key value :  dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR']) 


target 클래스의 값과 분포도
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64 


target 클래스의 이름들
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


이렇게 target 클래스의 이름들과 이에 해당하는 데이터가 dataset 내에 각 각 몇 개씩 있는지 확인하였다.<br><br>
그렇다면 각 data 가 어떻게 구성되고 있는지 살펴보자. 

In [3]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

기사 내용뿐만 아니라 보내는 사람, 제목, 작성자, 수신 이메일, 소속 등 다양한 정보를 내포하고 있다. <br><br>
텍스트 분석 개념을 다룰 때 말했듯이 텍스트 전처리 첫 번째로 기사 내용을 제외한 다른 정보는 모두 제거하는 __Cleansing 작업__ 을 수행 하자. <br><br>
fetch_20newsgroup 데이터 세트는 subset 파라미터로 이미 분리된 학습용, 테스트용 데이터 세트에 접근 할 수 있고, <br>
data 를 이미 부분별로 각 각 분리시켜 명칭이 부여 되어 있다. 그래서 기사 내용만 쉽게 추출해 낼 수 있다.

In [4]:
train_news = fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'),random_state=156)

X_train = train_news.data
y_train = train_news.target

test_news = fetch_20newsgroups(subset='test',remove=('headers','footers','quotes'),random_state=156)

X_test = test_news.data
y_test = test_news.target

print('학습 데이터 크기: {0}, 테스트 데이터 크기: {1}'.format(len(train_news.data),len(test_news.data)))

학습 데이터 크기: 11314, 테스트 데이터 크기: 7532


## 2. 피처 벡터화 & 3. ML 모델 학습 / 예측 / 평가

이제 피처 벡터화를 진행 해보자.<br><br>
먼저, CountVectorizer 를 이용해 학습 데이터의 텍스트를 피처 벡터화 한다. <br>
테스트 데이터 역시 동일하게 수행하는데 <span style="color:red">유의 할 점</span> 한 가지가 있다.
> 테스트 데이터에 CountVectorizer 을 적용 할 때는 반드시 학습 데이터를 이용해 fit() 이 수행된 객체를 이용해서 transform 해야 한다. <br><br>
그래야 학습 시 설정된 CountVectorizer 의 피처 개수와 테스트 데이터를 CountVectorizer 로 변환할 피처 개수가 같아진다. <br><br>
테스트 데이터에서 다시 fit() 을 하게 되면, 학습 데이터에 적용한 피처 벡터화의 결과와 다른 값을 새롭게 변환하는 꼴이 되어 버린다.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

# X_train 학습 데이터를 피처 벡터화
cnt_vec = CountVectorizer()
cnt_vec.fit(X_train)
X_train_cnt_vec = cnt_vec.transform(X_train)

# X_test 테스트 데이터를 피처 벡터화 
X_test_cnt_vec = cnt_vec.transform(X_test)

print("학습 데이터 텍스트의 CountVectorizer Shape: ",X_train_cnt_vec.shape)

학습 데이터 텍스트의 CountVectorizer Shape:  (11314, 101631)


학습 데이터를 CountVectorizer 로 피처를 추출한 결과, 11314개 문서에서 101,631 개의 단어를 추출하여 피처로 구성이 되었다. <br><br>
이렇게 구성된 데이터에 로지스틱 회귀를 적용해서 뉴스그룹에 대한 학습/예측/평가 를 해 보자. 

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vec,y_train)
pred = lr_clf.predict(X_test_cnt_vec)

print('CountVectorized Logistic Regression 예측 정확도 :{0:.3f}'.format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression 예측 정확도 :0.607


BOW 의 Count 기반으로 피처 벡터화가 적용된 데이터 세트에 대한 LogisticRegression 의 예측 정확도를 확인 했다.<br>
이번에는 TF-IDF 기반으로 벡터화한 뒤 예측 모델을 수행 해보자. 

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# X_train 학습 데이터를 피처 벡터화 
tfidf_vec = TfidfVectorizer()
tfidf_vec.fit(X_train)
X_train_tfidf_vec = tfidf_vec.transform(X_train)

# X_test 테스트 데이터를 피처 벡터화 
X_test_tfidf_vec = tfidf_vec.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vec,y_train)
pred_tfidf = lr_clf.predict(X_test_tfidf_vec)

print('TF-IDF Logistic Regression 예측 정확도: {0:.3f}'.format(accuracy_score(y_test,pred_tfidf)))

TF-IDF Logistic Regression 예측 정확도: 0.674


Count 기반 벡터화보다 TF-IDF 벡터화가 더 좋은 예측 결과를 도출했다. <br><br>
<span style="color:red">텍스트 분석에서 ML 모델의 성능을 향상시키는 중요한 2가지 방법은</span>
1. 최적의 ML 알고리즘 선택 
2. 최상의 피처 전처리 수행 

이다. <br><br>
즉, 두 방법으로 텍스트 기반의 머신러닝 성능에 큰 영향이 있다는 것이다. <br>
성능을 더 높이기 위해 TF-IDF 피처 벡터화의 파라미터를 조정해서 수행 해보자. <br><br>
[TF-IDF 공식 문서](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html?highlight=tfidf#sklearn.feature_extraction.text.TfidfVectorizer)



In [10]:
# stop words 필터링 , ngram_range 조정 , max_df 조정
tfidf_vec = TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=300)
tfidf_vec.fit(X_train)
X_train_tfidf_vec = tfidf_vec.transform(X_train)
X_test_tfidf_vec = tfidf_vec.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vec,y_train)
pred_tfidf = lr_clf.predict(X_test_tfidf_vec)

print("TF-IDF Logistic Regression (TF-IDF parameter adjustment) : {0:.3f}".format(accuracy_score(y_test,pred_tfidf)))

TF-IDF Logistic Regression (TF-IDF parameter adjustment) : 0.692


TF-IDF 피처 벡터화의 파라미터를 일부 조정한 결과 예측 정확도가 살짝 오른 것을 볼 수 있다. <br><br>
이번에는 GridSearchCV 를 이용해서 LogisticRegression 하이퍼 파라미터를 조정해 보자. 

In [17]:
from sklearn.model_selection import GridSearchCV
import time 

startTime = time.time()

params = {'C':[0.01,1,10]}

grid_cv_lr = GridSearchCV(lr_clf,param_grid=params,cv=3,scoring='accuracy',verbose=1)

grid_cv_lr.fit(X_train_tfidf_vec,y_train)

print("Logistic Regression best C parameter: ",grid_cv_lr.best_params_)
print("Logistic Regression best score: ",grid_cv_lr.best_score_)
pred = grid_cv_lr.predict(X_test_tfidf_vec)

print("TF-IDF Logistic Regression (TF-IDF parameter adjustment + lr parameter adjustment) : {0:.3f}".format(accuracy_score(y_test,pred)))

print("시간 측정: ",time.time()-startTime)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 17.6min finished


Logistic Regression best C parameter:  {'C': 10}
Logistic Regression best score:  0.752077537464524
TF-IDF Logistic Regression (TF-IDF parameter adjustment + lr parameter adjustment) : 0.701
시간 측정:  1247.7960860729218


약 20분 정도 소요 되었다. 약 만 개의 document 에서 약 10만 개의 피처를 추출한 데이터 세트에서 차원 축소도 없이 곧바로 GridSearchCV 를 적용
했기<br>때문에 당연히 오래 걸릴 수 밖에 없다.<br><br>
아무튼 LogisticRegression 모델의 C 파라미터가 10 일 때 검증 최고 정확도가 0.75로 나타났고 이를 기반으로 실제 예측을 진행 해본 결과 TF-IDF 하이퍼 파라미터만 조정했을 때의 예측 정확도 보다 약간의 성능이 향상 된 것을 확인 할 수 있다. 

### 사이킷런 Pipeline 사용 및 GridSearchCV 와의 결합

__< 사이킷런의 Pipeline 클래스란 ? >__ <br><br>
피처 벡터화와 ML 알고리즘 학습 / 예측을 위한 코드 작성을 한 번에 진행 할 수 있다. <br><br>
일반적으로 머신러닝에서 Pipeline 이란 <span style="color:red">데이터의 가공, 변환 등의 전처리 ( ex. 스케일링, 차원 변환 등 ) 와 ML 알고리즘 적용을</span> 마치 ' 수도관 (pipe) 에 물이 흐르듯 ' <span style="color:red">한꺼번에 스트림 기반으로 처리 한다</span> 는 의미를 갖는다. <br><br>
대용량 데이터의 피처 벡터화 결과를 별도 데이터로 저장하지 않고 바로 ML 알고리즘의 데이터로 입력 할 수 있어 수행 시간을 절약 할 수 있으며, 모든 데이터 전처리 작업과 Estimator 를 결합 할 수 있다. <br><br><br>
예제로 확인 해보자.<br><br>
__이 때, pipeline 내 객체 별도의 fit,transform 이나 fit,predict 를 어떻게 처리하는지 유의하자.__

In [11]:
from sklearn.pipeline import Pipeline

# Tfidvertorzier 객체를 tfidf_vec 라는 변수명으로 , LogisticRegression 객체를 lr_clf 라는 변수명으로 생성하는 Pipeline 생성
pipeline = Pipeline([
    ('tfidf_vec',TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=300)),
    ('lr_clf',LogisticRegression(C=10))
])

# 각 객체에 해당하는 fit,transform, predict 메서드를 사용하지 않고, pipeline 의 fit() 과 predict() 만으로 한꺼번에 처리 가능 
pipeline.fit(X_train,y_train)
pred = pipeline.predict(X_test)

print('Pipeline 을 통한 Logitic Regression 의 예측 정확도 : {0:.3f}'.format(accuracy_score(y_test,pred)))

Pipeline 을 통한 Logitic Regression 의 예측 정확도 : 0.701


사이킷런은 GridSearchCV 클래스의 생성 파라미터로 Pipeline 을 입력해 하이퍼 파라미터 튜닝 진행 할 수 있게 지원한다. <br><br>
다음 예제는 __GridSearchCV 에 Pipeline 을 입력하면서 TfidfVectorizer 의 파라미터와 LogisticRegression 의 하이퍼 파라미터를 함께 최적화 하는 방식__ <br> 으로 정확도를 예측 해 볼 것이다.
* 유의 할 점 <br><br>
GridSearchCV 에 Estimator 이 아닌 Pipeline 을 입력 할 경우, param_grid 의 입력 값 설정이 기존과 약간 다르다. <br>
<span style="color:red">구성하는 객체의 변수명 + __ + 조정 하려는 파라미터</span> 를 params_grid 의 입력 값의 Key 값으로 설정 해주어야 한다. <br>
다음 예제를 보면 이해가 쉽다. <br><br>

* 진행 중인 과정 중간 체크<br>

> Pipeline 으로 텍스트 전처리 + ML 알고리즘 적용을 동시에 하면서 GridSearchCV 로 각 클래스 파라미터들을 조정 해주는 과정 진행 중 ... 

In [12]:
from sklearn.model_selection import GridSearchCV
import time 

startTime = time.time() 

pipeline = Pipeline([
    ('tfidf_vec',TfidfVectorizer(stop_words='english')),
    ('lr_clf',LogisticRegression())
])

# 유의 할 부분 : GridSearchCV 에 Pipeline 이 입력 되기 때문에 위 설명에 맞게 params_grid 입력값에서 Key 값 설정하기 

params = {'tfidf_vec__ngram_range':[(1,1),(1,3)],
         'tfidf_vec__max_df':[100,700],
         'lr_clf__C':[1,10]
         }
# GridSearchCV 의 생성자에 Estimator 이 아닌 Pipeline 객체 입력 
grid_cv_pipe = GridSearchCV(pipeline,param_grid=params,cv=2,scoring='accuracy',verbose=1,n_jobs=-1)
grid_cv_pipe.fit(X_train,y_train)

print('Pipeline 을 통한 GridSearchCV 최적 파라미터: ',grid_cv_pipe.best_params_)
print('Pipeline 을 통한 GridSearchCV 검증 최고 정확도: {0:.3f}'.format(grid_cv_pipe.best_score_))

pred_pipe = grid_cv_pipe.predict(X_test)
print('Pipeline 을 통한 GridSearchCV 예측 정확도: {0:.3f}'.format(accuracy_score(y_test,pred_pipe)))

print('시간 측정: ',time.time()-startTime)

Fitting 2 folds for each of 8 candidates, totalling 16 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  16 | elapsed:  1.5min remaining: 10.5min
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed: 24.3min finished


Pipeline 을 통한 GridSearchCV 최적 파라미터:  {'lr_clf__C': 10, 'tfidf_vec__max_df': 700, 'tfidf_vec__ngram_range': (1, 1)}
Pipeline 을 통한 GridSearchCV 검증 최고 정확도: 0.735
Pipeline 을 통한 GridSearchCV 예측 정확도: 0.694
시간 측정:  1482.4292311668396


조정하고자 하는 파라미터 3 개를 각 2개씩 총 8개의 파라미터 경우의 수를, 검증을 위해 2 개로 분할을 했으니 총 16번의 학습과 검증을 수행하였고<br>그 결과 약 25분의 수행 시간이 걸렸다. ( 심지어, 병렬처리하여 모든 코어를 동원 시키기 위해 n_job 도 조정 했다 )<br><br>
이처럼 Pipeline + GridSearchCV 를 적용 할 때, 여러 파라미터를 최적화하려면 너무 많은 튜닝 시간이 소모되는 것을 유의 해야 한다. <br><br>
이제 결과를 보자.<br>
최적화한 파라미터를 기반으로 예측을 해도 정확도는 크게 개선 되지 않았다. <br>
성능 개선을 위해 Logistic Regression 외에, 희소 행렬 기반의 텍스트 분류에 자주 사용되는 Support Vector Machine 이나 Naive Bayes 를 이용 해보자. 